In [948]:
import numpy as np
import pandas as pd
import time

In [949]:
from library_w19_deep_2 import *

%who function

ann_flex	 ann_flex_predictor	 ann_flex_tester	 ann_predictor	 ann_simple	 ann_simple_batch	 ann_tester	 from_scratch	 from_scratch_batch	 
from_scratch_flex	 graph	 mse	 mse_der	 sigmoid	 sigmoid_der	 


In [950]:
with open('/Users/Noah/Documents/My Documents/2018:2019/2. Winter/CIS 399/Project/tracks.csv', 'r') as f:
    temp_table = pd.read_csv(f)


/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,48,49,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [951]:
track_table = temp_table.copy()#for testing
track_table.head()
#Lots of Garbage Columns and Empty Rows
#Not all Columns have a uniform type yet

,track_id,comments,date_created,date_released,engineer,favorites,id,information,listens,producer,...,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103
0,2,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,0,11/26/08 1:44,1/5/09 0:00,NaN,4,1,<p></p>,6073,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,0,11/26/08 1:45,2/6/08 0:00,NaN,4,6,NaN,47632,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20,0,11/26/08 1:45,1/6/09 0:00,NaN,2,4,"<p>¬†""spiritual songs"" from Nicky Cook</p>",2710,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [952]:
"""Data Wrangling"""
#Drop useless columns & Clean up names
track_table = track_table.drop(columns=['comments','date_created','engineer','favorites','id','information','producer','tracks','type','tags','active_year_begin','active_year_end','associated_labels','bio','comments.1','date_created.1','favorites.1','id.1','latitude','longitude','members','related_projects','tags.1','website','wikipedia_page','split','subset','comments.2','composer','date_created.2','date_recorded','favorites.2','genres','genres_all','information','interest','license','lyricist','number','publisher','tags','information.1','tags.2'])
track_table = track_table.drop(track_table.columns[12:], axis=1)
track_table = track_table.rename(index=str, columns={"listens": "album_listens", "title": "album", "name": "artist", "genre_top": "genre", "listens.1": "listens", "title.1": "title"})

#Make sure all ints. If not possible, makes NaN
track_table['album_listens'] = pd.to_numeric(track_table['album_listens'], errors='coerce')
track_table['listens'] = pd.to_numeric(track_table['listens'], errors='coerce')
track_table['bit_rate'] = pd.to_numeric(track_table['bit_rate'], errors='coerce')
track_table['duration'] = pd.to_numeric(track_table['duration'], errors='coerce')

#Could expand to include other languages
track_table.language_code[track_table.language_code != 'en'] = 0
track_table.language_code[track_table.language_code == 'en'] = 1

#Convert genres to ints
col = []
for i in range(len(track_table)):
    if(track_table['genre'][i]=='Hip-Hop'):
        col.append(1)
    elif(track_table['genre'][i]=='Experimental'):
        col.append(2)
    elif(track_table['genre'][i]=='Folk'):
        col.append(3)
    elif(track_table['genre'][i]=='Jazz'):
        col.append(4)
    elif(track_table['genre'][i]=='Rock'):
        col.append(5)
    elif(track_table['genre'][i]=='Pop'):
        col.append(6)
    elif(track_table['genre'][i]=='Electronic'):
        col.append(7)
    elif(track_table['genre'][i]=='International'):
        col.append(8)
    elif(track_table['genre'][i]=='Soul-RnB'):
        col.append(9)
    elif(track_table['genre'][i]=='Spoken'):
        col.append(10)
    elif(track_table['genre'][i]=='Blues'):
        col.append(11)
    elif(track_table['genre'][i]=='Country'):
        col.append(12)
    elif(track_table['genre'][i]=='Instrumental'):
        col.append(13)
    elif(track_table['genre'][i]=='Classical'):
        col.append(14)
    elif(track_table['genre'][i]=='Easy Litening'):
        col.append(15)
    else:
        col.append(0)
track_table['genre'] = col

track_table = track_table.dropna()
track_table.head()

/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,track_id,date_released,album_listens,album,location,artist,bit_rate,duration,genre,language_code,listens,title
0,2,1/5/09 0:00,6073.0,AWOL - A Way Of Life,New Jersey,AWOL,256000.0,168.0,1,1,1293.0,Food
1,3,1/5/09 0:00,6073.0,AWOL - A Way Of Life,New Jersey,AWOL,256000.0,237.0,1,1,514.0,Electric Ave
2,5,1/5/09 0:00,6073.0,AWOL - A Way Of Life,New Jersey,AWOL,256000.0,206.0,1,1,1151.0,This World
4,20,1/6/09 0:00,2710.0,Niris,Colchester England,Nicky Cook,256000.0,311.0,0,1,361.0,Spiritual Level
5,26,1/6/09 0:00,2710.0,Niris,Colchester England,Nicky Cook,256000.0,181.0,0,1,193.0,Where is your Love?


In [953]:
"""More Data Wrangling"""
month = []
day = []
year = []
seconds = []
for i in range(len(track_table)):
    date = str(track_table['date_released'][i][:-5])
    date = date.split('/')
    month.append(int(date[0]))
    day.append(int(date[1]))
    year.append(int(date[2]))
    #Convert date to something useful
    if(int(date[2])<19):
        y = 2000+int(date[2])
    else:
        y = 1900+int(date[2])
    #Seconds Since Epoch
    t = (y, int(date[0]), int(date[1]),0,0,0,0,0,0)
    seconds.append(time.mktime(t))
                           
track_table['day'] = day
track_table['month'] = month
track_table['year'] = year
track_table['seconds'] = seconds

#OHE Labels
bins = pd.IntervalIndex.from_tuples([(0,250),(250,500),(500,1000),(1000,10000),(10000,1000000)])
track_table['listens_binned'] = pd.cut(track_table['listens'], bins)
one_hot_listens = pd.get_dummies(track_table["listens_binned"], prefix = "listens", dummy_na = False)
track_table = track_table.join(one_hot_listens)
labels = track_table[['listens_(0, 250]','listens_(250, 500]','listens_(500, 1000]','listens_(1000, 10000]','listens_(10000, 1000000]']].values

#Normalize
track_table["album_listens_normed"] = track_table["album_listens"] / track_table["album_listens"].max()
track_table["bit_rate_normed"] = track_table["bit_rate"] / track_table["bit_rate"].max()
track_table["duration_normed"] = track_table["duration"] / track_table["duration"].max()
track_table["genre_normed"] = track_table["genre"] / track_table["genre"].max()
track_table["seconds_normed"] = track_table["seconds"] / track_table["seconds"].max()
track_table["month_normed"] = track_table["month"] / track_table["month"].max()
track_table["year_normed"] = track_table["year"] / track_table["year"].max()

track_table.head()

,track_id,date_released,album_listens,album,location,artist,bit_rate,duration,genre,language_code,...,"listens_(500, 1000]","listens_(1000, 10000]","listens_(10000, 1000000]",album_listens_normed,bit_rate_normed,duration_normed,genre_normed,seconds_normed,month_normed,year_normed
0,2,1/5/09 0:00,6073.0,AWOL - A Way Of Life,New Jersey,AWOL,256000.0,168.0,1,1,...,0,1,0,0.001704,0.764375,0.009155,0.071429,0.813256,0.083333,0.090909
1,3,1/5/09 0:00,6073.0,AWOL - A Way Of Life,New Jersey,AWOL,256000.0,237.0,1,1,...,1,0,0,0.001704,0.764375,0.012916,0.071429,0.813256,0.083333,0.090909
2,5,1/5/09 0:00,6073.0,AWOL - A Way Of Life,New Jersey,AWOL,256000.0,206.0,1,1,...,0,1,0,0.001704,0.764375,0.011226,0.071429,0.813256,0.083333,0.090909
4,20,1/6/09 0:00,2710.0,Niris,Colchester England,Nicky Cook,256000.0,311.0,0,1,...,0,0,0,0.000760,0.764375,0.016948,0.000000,0.813313,0.083333,0.090909
5,26,1/6/09 0:00,2710.0,Niris,Colchester England,Nicky Cook,256000.0,181.0,0,1,...,0,0,0,0.000760,0.764375,0.009864,0.000000,0.813313,0.083333,0.090909


In [954]:
"""
Testing with all features yielded low accuracy. 
I looked at weights and accuracies and dropped features with low impact
"""

#ann_table = track_table[['album_listens_normed', 'bit_rate_normed', 'duration_normed', 'language_code','genre_normed','seconds_normed','year_normed','month_normed']]
#album_listens_normed: .59
#album_listens_normed, bit_rate_normed: .59
#[['album_listens_normed','bit_rate_normed','duration_normed','genre_normed','day']]: .435
#[['album_listens_normed','bit_rate_normed','duration_normed','genre_normed','month']]: .625
#[['album_listens_normed','bit_rate_normed','duration_normed','genre_normed','day']]: .435
#track_table[['album_listens_normed','bit_rate_normed','duration_normed', 'language_code','genre_normed','month']]: .65
#[['album_listens_normed','bit_rate_normed','duration_normed', 'language_code','genre_normed','month','year']]: .23
#[['album_listens_normed','bit_rate_normed','duration_normed', 'language_code','genre_normed','seconds_normed', 'month_normed','year_normed']]: .655
#[['album_listens_normed','bit_rate_normed','duration_normed', 'language_code','genre_normed','seconds_normed', 'month_normed']]: .675
ann_table = track_table[['album_listens_normed','bit_rate_normed','duration_normed', 'language_code','genre_normed','seconds_normed', 'month_normed']]
feature_set = ann_table.values

print(labels.shape)
print(feature_set.shape)

ann_table.head()

(48166, 5)
(48166, 7)


,album_listens_normed,bit_rate_normed,duration_normed,language_code,genre_normed,seconds_normed,month_normed
0,0.001704,0.764375,0.009155,1,0.071429,0.813256,0.083333
1,0.001704,0.764375,0.012916,1,0.071429,0.813256,0.083333
2,0.001704,0.764375,0.011226,1,0.071429,0.813256,0.083333
4,0.000760,0.764375,0.016948,1,0.000000,0.813313,0.083333
5,0.000760,0.764375,0.009864,1,0.000000,0.813313,0.083333


In [964]:
"""Using ann_flex"""
samples = 250 #Number of samples to train with
(weights, biases) = from_scratch_flex(feature_set[:samples], labels[:samples], hypers={'epochs':10000, 'cost-reporting': 1000})#, 'learning-rate': .05})
print(weights)
print(biases)

(0, 0.5377388405810963)
(1000, 0.5201852527678641)
(2000, 0.5180548206136583)
(3000, 0.5177054759068811)
(4000, 0.5176498921412406)
(5000, 0.5175715246039947)
(6000, 0.5174464885161064)
(7000, 0.5172904698121925)
(8000, 0.5171180760979738)
(9000, 0.5169385590670066)
(10000, 0.5167572363252897)
[[-50.66874701839435 0.5551522774011513 18.286064048060858
  0.23882276196558364 -0.07161064413042505]
 [-7.389559735401236 5.763486408769449 7.225464434935459
  3.7027518926067455 -1.8335284278371768]
 [-19.12349001100998 39.225553337532936 -59.69726962982907
  4.467894491303452 -0.10273105860212838]
 [2.292236724079512 -2.0639898991098797 -3.1755482438523064
  -2.9503080619739785 -2.904053391160932]
 [3.074275253886367 -3.55105687648708 -7.077389467489342
  -0.9071308284518803 -0.6642402888526816]
 [-0.6674783587537468 -2.720003451981374 1.571298413913735
  0.8410128959827128 -2.136291603241677]
 [1.4241665340689986 0.12354171502207581 -2.702500344790676
  -2.8167961066694422 -0.372553170558263

In [965]:
predictions = ann_flex_tester(feature_set[:samples], weights, biases)
accuracy = 0
prediction_list = []

for k in range(len(predictions)):
    
    #Find index of largest value in list
    mx = index_min = np.argmax(predictions[k].tolist())
    #Set index of largest to 1, all else to 0
    pred = [0,0,0,0,0]
    pred[mx] = 1
    prediction_list.append(pred)


for j in range(len(prediction_list)):
    #Compare Prediction to Target Label
    if(prediction_list[j] == list(labels[j])):
        accuracy += 1

print("*"*64)
accuracy = accuracy / len(prediction_list)
print("Accuracy: %s"%(accuracy))

"""
The best accuracy I could get is .675, just over 2/3. 
Different Alphas didn't seem to have a huge effect.
I think part of the problem is that the dataset I used wasn't designed for this kind of testing, 
and the columns I used to decide the listens just aren't that indicative of how popular a song will be.
"""

****************************************************************
Accuracy: 0.632


"\nThe best accuracy I could get is .675, just over 2/3. \nI think part of the problem is that the dataset I used wasn't designed for this kind of testing, \nand the columns I used to decide the listens just aren't that indicative of how popular a song will be.\n"